In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re

In [2]:
# step 2
# Загрузите данные, проверьте типы, пропущенные значения. 
# Сколько пропущенных значений встретилось в датасете?
cars = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-o-bogdanova/stat_lesson7/cars.csv')
cars

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


In [3]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [4]:
cars.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [5]:
# step 3
# Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). 
# Для этого используйте столбец CarName, разбейте значения ячеек по пробелу и запишите в колонку первый элемент. Например:
# 'audi 100 ls' → 'audi'
# Сколько всего уникальных марок машины встречается в датасете? 
cars['company'] = cars['CarName'] \
    .str.split(' ') \
    .str[0]
# Сколько всего уникальных марок машины встречается в датасете? 
cars.company.nunique()

28

In [6]:
# Столбец CarName с полным названием машины удалите из датасета, а также car_ID, они не пригодятся для дальнейшего анализа.
cars = cars.drop(columns = ['CarName', 'car_ID'])
cars.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


In [7]:
# step 4
# Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками!
cars.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

In [8]:
# замена ошибочных вариантов:
        # создание паттернов регулярных выражений 
pattern_mazda = re.compile('ma\wda')
pattern_nissan = re.compile('\wissan')
pattern_porsche = re.compile('porc?s?c?hc?e')
pattern_toyota = re.compile('toyou?ta')
pattern_volkswagen = re.compile('v\w*w\w*')

In [9]:
cars.replace(pattern_mazda, 'mazda', inplace = True)
cars.replace(pattern_nissan, 'nissan', inplace = True)
cars.replace(pattern_porsche, 'porsche', inplace = True)
cars.replace(pattern_toyota, 'toyota', inplace = True)
cars.replace(pattern_vokswagen, 'volkswagen', inplace = True)

NameError: name 'pattern_vokswagen' is not defined

In [10]:
cars.company.nunique()

24

In [11]:
# step 5
# Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:
# 'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'
# также не забыв про то, что мы предсказываем – 'price'. 
# После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower? 
# Ответ округлите до 2 знаков после точки.
for_model = cars.copy() \
    [['company', 
      'fueltype', 
      'aspiration',
      'carbody', 
      'drivewheel', 
      'wheelbase', 
      'carlength',
      'carwidth', 
      'curbweight', 
      'enginetype', 
      'cylindernumber', 
      'enginesize', 
      'boreratio',
      'horsepower', 
      'price']]
for_model

,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower,price
0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,13495.0
1,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,16500.0
2,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154,16500.0
3,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102,13950.0
4,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,2952,ohc,four,141,3.78,114,16845.0
201,volvo,gas,turbo,sedan,rwd,109.1,188.8,68.8,3049,ohc,four,141,3.78,160,19045.0
202,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,3012,ohcv,six,173,3.58,134,21485.0
203,volvo,diesel,turbo,sedan,rwd,109.1,188.8,68.9,3217,ohc,six,145,3.01,106,22470.0


In [12]:
for_model.corr()

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294,0.577816
carlength,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623,0.682920
carwidth,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732,0.759325
curbweight,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739,0.835305
enginesize,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769,0.874145
boreratio,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677,0.553173
horsepower,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000,0.808139
price,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139,1.000000


In [13]:
# step 6
# Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), 
# поэтому давайте применим функцию под названием pd.get_dummies()

df_dummy = pd.get_dummies(data=for_model[['fueltype', 
                                     'aspiration', 
                                    'carbody',
                                    'drivewheel', 
                                    'enginetype',
                                    'cylindernumber']], drop_first = True)

In [14]:
dummy_for_model = pd.get_dummies(data=for_model, drop_first = True)
dummy_for_model

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,88.6,168.8,64.1,2548,130,3.47,111,13495.0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,88.6,168.8,64.1,2548,130,3.47,111,16500.0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,94.5,171.2,65.5,2823,152,2.68,154,16500.0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,99.8,176.6,66.2,2337,109,3.19,102,13950.0,1,0,...,1,0,0,0,0,1,0,0,0,0
4,99.4,176.6,66.4,2824,136,3.19,115,17450.0,1,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,109.1,188.8,68.9,2952,141,3.78,114,16845.0,0,0,...,1,0,0,0,0,1,0,0,0,0
201,109.1,188.8,68.8,3049,141,3.78,160,19045.0,0,0,...,1,0,0,0,0,1,0,0,0,0
202,109.1,188.8,68.9,3012,173,3.58,134,21485.0,0,0,...,0,0,1,0,0,0,1,0,0,0
203,109.1,188.8,68.9,3217,145,3.01,106,22470.0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [15]:
# step 7
#  Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.
# Какой процент изменчивости объясняет полученная модель? (округлите до целого)
Y = dummy_for_model['price']
X = dummy_for_model['horsepower']
X = sm.add_constant(X)
model1 = sm.OLS(Y,X)
results1 = model.fit()
print(results1.summary())

NameError: name 'model' is not defined

In [16]:
# step 8
# Теперь построим ещё две модели:
# модель со всеми предикторами
# модель со всеми предикторами, кроме марок машин
# Попробуем выбрать из этих моделей самую удачную! В каком-то смысле это искусство - 
# при выборе моделей приходится ориентироваться не только на количественные показатели, но также на контекст её использования, опыт и здравый смысл.

In [21]:
column_list = list(dummy_for_model.columns)
for_formula_1 = ''
for i in column_list:
    if i!='price':
        for_formula_1+= 'dummy_for_model.' + str(*[i]) + '+'
for_formula_1

'dummy_for_model.wheelbase+dummy_for_model.carlength+dummy_for_model.carwidth+dummy_for_model.curbweight+dummy_for_model.enginesize+dummy_for_model.boreratio+dummy_for_model.horsepower+dummy_for_model.company_audi+dummy_for_model.company_bmw+dummy_for_model.company_buick+dummy_for_model.company_chevrolet+dummy_for_model.company_dodge+dummy_for_model.company_honda+dummy_for_model.company_isuzu+dummy_for_model.company_jaguar+dummy_for_model.company_mazda+dummy_for_model.company_mercury+dummy_for_model.company_mitsubishi+dummy_for_model.company_nissan+dummy_for_model.company_peugeot+dummy_for_model.company_plymouth+dummy_for_model.company_porsche+dummy_for_model.company_renault+dummy_for_model.company_saab+dummy_for_model.company_subaru+dummy_for_model.company_toyota+dummy_for_model.company_vokswagen+dummy_for_model.company_volkswagen+dummy_for_model.company_volvo+dummy_for_model.company_vw+dummy_for_model.fueltype_gas+dummy_for_model.aspiration_turbo+dummy_for_model.carbody_hardtop+dummy

In [18]:
# модель со всеми предикторами
results2 = smf.ols('dummy_for_model.price ~ dummy_for_model.wheelbase+dummy_for_model.carlength+dummy_for_model.carwidth+dummy_for_model.curbweight+dummy_for_model.enginesize+dummy_for_model.boreratio+dummy_for_model.horsepower+dummy_for_model.company_audi+dummy_for_model.company_bmw+dummy_for_model.company_buick+dummy_for_model.company_chevrolet+dummy_for_model.company_dodge+dummy_for_model.company_honda+dummy_for_model.company_isuzu+dummy_for_model.company_jaguar+dummy_for_model.company_mazda+dummy_for_model.company_mercury+dummy_for_model.company_mitsubishi+dummy_for_model.company_nissan+dummy_for_model.company_peugeot+dummy_for_model.company_plymouth+dummy_for_model.company_porsche+dummy_for_model.company_renault+dummy_for_model.company_saab+dummy_for_model.company_subaru+dummy_for_model.company_toyota+dummy_for_model.company_volkswagen+dummy_for_model.company_volvo+dummy_for_model.fueltype_gas+dummy_for_model.aspiration_turbo+dummy_for_model.carbody_hardtop+dummy_for_model.carbody_hatchback+dummy_for_model.carbody_sedan+dummy_for_model.carbody_wagon+dummy_for_model.drivewheel_fwd+dummy_for_model.drivewheel_rwd+dummy_for_model.enginetype_dohcv+dummy_for_model.enginetype_l+dummy_for_model.enginetype_ohc+dummy_for_model.enginetype_ohcf+dummy_for_model.enginetype_ohcv+dummy_for_model.enginetype_rotor+dummy_for_model.cylindernumber_five+dummy_for_model.cylindernumber_four+dummy_for_model.cylindernumber_six+dummy_for_model.cylindernumber_three+dummy_for_model.cylindernumber_twelve+dummy_for_model.cylindernumber_two', dummy_for_model).fit()
print(results2.summary())

                              OLS Regression Results                             
Dep. Variable:     dummy_for_model.price   R-squared:                       0.959
Model:                               OLS   Adj. R-squared:                  0.947
Method:                    Least Squares   F-statistic:                     80.54
Date:                   Sun, 15 Jan 2023   Prob (F-statistic):           8.07e-89
Time:                           22:33:55   Log-Likelihood:                -1804.8
No. Observations:                    205   AIC:                             3704.
Df Residuals:                        158   BIC:                             3860.
Df Model:                             46                                         
Covariance Type:               nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

In [22]:
for_formula_2 = ''
for i in column_list:
    if i!='price':
        if i.startswith('company') is False: 
            for_formula_2 += 'dummy_for_model.' + str(*[i]) + '+'
for_formula_2

'dummy_for_model.wheelbase+dummy_for_model.carlength+dummy_for_model.carwidth+dummy_for_model.curbweight+dummy_for_model.enginesize+dummy_for_model.boreratio+dummy_for_model.horsepower+dummy_for_model.fueltype_gas+dummy_for_model.aspiration_turbo+dummy_for_model.carbody_hardtop+dummy_for_model.carbody_hatchback+dummy_for_model.carbody_sedan+dummy_for_model.carbody_wagon+dummy_for_model.drivewheel_fwd+dummy_for_model.drivewheel_rwd+dummy_for_model.enginetype_dohcv+dummy_for_model.enginetype_l+dummy_for_model.enginetype_ohc+dummy_for_model.enginetype_ohcf+dummy_for_model.enginetype_ohcv+dummy_for_model.enginetype_rotor+dummy_for_model.cylindernumber_five+dummy_for_model.cylindernumber_four+dummy_for_model.cylindernumber_six+dummy_for_model.cylindernumber_three+dummy_for_model.cylindernumber_twelve+dummy_for_model.cylindernumber_two+'

In [23]:
# модель со всеми предикторами, кроме марок машин

results3 = smf.ols('dummy_for_model.price ~ dummy_for_model.wheelbase+dummy_for_model.carlength+dummy_for_model.carwidth+dummy_for_model.curbweight+dummy_for_model.enginesize+dummy_for_model.boreratio+dummy_for_model.horsepower+dummy_for_model.fueltype_gas+dummy_for_model.aspiration_turbo+dummy_for_model.carbody_hardtop+dummy_for_model.carbody_hatchback+dummy_for_model.carbody_sedan+dummy_for_model.carbody_wagon+dummy_for_model.drivewheel_fwd+dummy_for_model.drivewheel_rwd+dummy_for_model.enginetype_dohcv+dummy_for_model.enginetype_l+dummy_for_model.enginetype_ohc+dummy_for_model.enginetype_ohcf+dummy_for_model.enginetype_ohcv+dummy_for_model.enginetype_rotor+dummy_for_model.cylindernumber_five+dummy_for_model.cylindernumber_four+dummy_for_model.cylindernumber_six+dummy_for_model.cylindernumber_three+dummy_for_model.cylindernumber_twelve+dummy_for_model.cylindernumber_two', dummy_for_model).fit()
print(results3.summary())

                              OLS Regression Results                             
Dep. Variable:     dummy_for_model.price   R-squared:                       0.914
Model:                               OLS   Adj. R-squared:                  0.901
Method:                    Least Squares   F-statistic:                     72.32
Date:                   Sun, 15 Jan 2023   Prob (F-statistic):           9.86e-81
Time:                           22:43:01   Log-Likelihood:                -1881.6
No. Observations:                    205   AIC:                             3817.
Df Residuals:                        178   BIC:                             3907.
Df Model:                             26                                         
Covariance Type:               nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------